<a href="https://colab.research.google.com/github/matheusfalango/RAIA-fellowship/blob/main/Classify-DayNight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAIA Fellowship 2026.1
## Problema 1 - Clasificação de imagens em dia ou noite utilizando Python.

---

### Contextualização do problema
A partir de um dataset (30 imagens) com paisagens em abientes externos sob diferentes condições de iluminação, clima, ambiente e saturação; deve-se elaborar uma solução em python para classificação da imagem em "dia" ou "noite" sob, pelo menos, duas abordagens distintas.

### Sugestões de abordagem

* Métricas das imagens referente a iluminação: brilho, saturação, contraste, nitidez;
* Características visuais básicas: coloração e itensidade da luz;
* Técnicas de Visão Computacional: comparação das métricas, histogramas.

### Discussão e Limitações

Para implementação de métodos de aprendizado de máquina, o dataset disponível é pequeno; podendo causar erros na classificação, tendo como fonte - por exemplo - iluminação artificial.

Para métricas das imagens, as fotos de saturação baixa devido à "poluição" visual do ambiente (embaçado, prédios, ambiente urbano) é uma dificuldade notável para classificação entre "dia" e "noite".

Além disso, outro tópico que influencia diretamente sobre a classificação é o clima; para paisagens em tempo nublado, a saturação e brilho, consequentemente, serão mais baixas, mesmo que a foto tenha sida capturada durante o dia.

Por fim, a última limitação observada é o horário; as paisagens mais próximas ao amanhecer ou entardecer estão em uma faixa de "transição" da classificação; podendo apresentar ambiguidade na decisão.

### Conclusão
Teoricamente, a classificação das paisagens em dia/noite podem ser definidas a partir de métricas básicas das imagens, como brilho, saturação, nitidez, contraste observando a escala de cores RGB e/ou cinza. Mas, é necessário ter atenção quanto a poluição visual, iluminação artificial, clima e horário das imagens capturadas presentes no dataset; pois são limites que a classificação contém.

# Base do código


In [ ]:
# Leitura de imagem

import cv2 as cv
from google.colab.patches import cv2_imshow

images = [] # Inicializa a lista de vetores das imagens em BGR

for i in range(0, 30):
  # Define o nome do arquivo a ser acessado
  filename = f'foto{i+1}.jpg'
  current_image = cv.imread(filename)

  # Verificar se a imagem foi carregada corretamente
  if current_image is None:
      break

  else:
      images.append(current_image) # Adiciona a imagem carregada à lista

print(f"Foram carregadas {len(images)} imagens.")

Foram carregadas 30 imagens.


# Abordagem 1 - Iluminação


## Resumo
Esta abordagem foca em classificar as imagens pelas métricas de luminosidade, a partir das escalas cinza e HSV (matiz, saturação e brilho). Logo, assumiu-se que as paisagens noturnas apresentam menos brilho, menor saturação e mais segmentos escuros.

## Motivação
As métricas de luminosidade para diferenciação entre dia/noite são diretas para a decisão, baseadas em lógica e interpretação.

## Pipeline
1. Leitura da imagem
2. Conversão para escala cinza e HSV
3. Obtenção das métricas de luminosidade
4. Classificação por heurística

## Limitações
Tópicos que interferem diretamente sobre a classificação: contraste forte devido à presença de sombras e ambientes mais fechados que apresentam pouca diversidade de tonalidade.

In [ ]:
import numpy as np

# Labels das métricas de iluminação (brilho, contraste, intensidade, saturação)
def features_illumination(image):

  gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
  hsv_image = cv.cvtColor(image, cv.COLOR_BGR2HSV)

  mean_brightness = np.mean(gray_image)
  std_brightness = np.std(gray_image)
  p10 = np.percentile(gray_image, 10)
  p90 = np.percentile(gray_image, 90)
  mean_saturation = np.mean(hsv_image[:, :, 1])

  return {
        "mean_brightness": mean_brightness,
        "std_brightness": std_brightness,
        "p10": p10,
        "p90": p90,
        "mean_saturation": mean_saturation
    }

In [ ]:
# Regras de classificação

def classify_illumination(feat, stats):

    shadow_range = feat["p90"] - feat["p10"]

    # Muito escuro
    if feat["mean_brightness"] < stats["brightness_low"]:
        return "noite"

    # Muito claro
    if feat["mean_brightness"] > stats["brightness_high"]:
        return "dia"

    # Iluminação artificial noturna
    if (
        feat["mean_saturation"] < stats["saturation_low"] and
        feat["std_brightness"] < stats["contrast_low"]
    ):
        return "noite"

    # Presença de sombra
    if shadow_range < stats["shadow_low"]:
        return "noite"

    if shadow_range > stats["shadow_high"]:
        return "dia"

    ### Fallback com peso maior no brilho ###

    night_score = 0
    day_score = 0

    #Brilho: mais perto de "escuro" ou "claro"?
    if abs(feat["mean_brightness"] - stats["brightness_low"]) < \
       abs(feat["mean_brightness"] - stats["brightness_high"]):
        night_score += 2
    else:
        day_score += 2

    # Saturação: mais perto do limiar noturno ou do mediano?
    if abs(feat["mean_saturation"] - stats["saturation_low"]) < \
       abs(feat["mean_saturation"] - stats["saturation_mid"]):
        night_score += 1
    else:
        day_score += 1

    # Alcance tonal: mais homogêneo ou mais contrastado?
    if abs(shadow_range - stats["shadow_low"]) < \
       abs(shadow_range - stats["shadow_high"]):
        night_score += 1
    else:
        day_score += 1

    return "noite" if night_score > day_score else "dia"


In [ ]:
# Análise das métricas de brilho, dark ratio, saturação, distribuição de brilho
import pandas as pd

illum_features = []

for i, img in enumerate(images):
    feat = features_illumination(cv.resize(img, (256, 256)))
    feat["imagem"] = f"foto{i+1}.jpg"
    illum_features.append(feat)

df_illum = pd.DataFrame(illum_features)
df_illum


,mean_brightness,std_brightness,p10,p90,mean_saturation,imagem
0,111.380219,51.737416,66.0,191.0,169.167450,foto1.jpg
1,123.678421,66.683564,15.0,183.0,69.765778,foto2.jpg
2,48.246567,41.550983,9.0,105.0,183.676651,foto3.jpg
3,124.393158,64.672421,28.0,201.0,91.411606,foto4.jpg
4,48.505081,31.216121,18.0,80.0,184.257828,foto5.jpg
5,125.800949,70.343259,33.0,206.0,54.159653,foto6.jpg
6,34.903381,46.967118,3.0,92.0,108.783630,foto7.jpg
7,125.803497,40.325103,81.0,180.0,115.372162,foto8.jpg
8,53.898376,64.977318,1.0,157.0,148.201370,foto9.jpg
9,66.211868,44.278021,13.0,113.0,131.410538,foto10.jpg


# Abordagem 2 - Cor e Saturação


## Resumo
Esta abordagem foca em classificar as imagens pelas cores e saturação da imagem; justifica-se pela tendência de imagens noturnas terem cores de tons menos vibrantes e iluminação artificial.

## Motivação
As métricas de luminosidade para diferenciação entre dia/noite são diretas para a decisão sob influência de iluminação artificial, logo complementaria a abordagem 1 por sofrer influência do tipo de iluminação da imagem na decisão final.

## Pipeline
1. Leitura da imagem
2. Conversão para escala HSV
3. Obtenção das métricas de cor/saturação
4. Classificação por regras cromáticas

## Limitações
Tópicos que interferem diretamente sobre a classificação: sensibilidade a cor branca devido a saturação e tom cromático.

In [ ]:
import numpy as np

# Labels das métricas de iluminação (saturação, tonalidade, proporção RG e BG)
def features_color(image):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    b, g, r = cv.split(image)

    mean_s = np.mean(hsv[:, :, 1])
    mean_h = np.mean(hsv[:, :, 0])

    rg_ratio = np.mean(r) / (np.mean(g) + 1e-6)
    bg_ratio = np.mean(b) / (np.mean(g) + 1e-6)

    return {
        "mean_saturation": mean_s,
        "mean_hue": mean_h,
        "rg_ratio": rg_ratio,
        "bg_ratio": bg_ratio
    }


In [ ]:
# Classificação heurística baseada nas métricas

def classify_color(feat, stats):
    s = feat["mean_saturation"]
    rg = feat["rg_ratio"]
    bg = feat["bg_ratio"]
    h = feat["mean_hue"]

    day_score = 0
    night_score = 0

    # Análise de saturação
    if s >= stats["saturation_high"]:
        day_score += 3

    elif s < stats["saturation_low"]:
        night_score += 3

    else:
        night_score += 1

    # Análise de balanço de cores
    color_balanced = (rg <= stats["rg_high"] and bg <= stats["bg_high"])

    if color_balanced:
        day_score += 2

    else:
        night_score += 2

    # Análise de tonalidade
    if stats["hue_low"] <= h <= stats["hue_high"]:
        day_score += 1

    else:
        night_score += 1

    # Decisão
    return "dia" if day_score > night_score else "noite"


In [ ]:
# Análise das métricas de cor: saturação, tonalidade e relações RGB
import pandas as pd

color_features = []

for i, img in enumerate(images):
    # Padroniza o tamanho para evitar viés
    feat = features_color(cv.resize(img, (256, 256)))

    # Nome da imagem
    feat["imagem"] = f"foto{i+1}.jpg"

    color_features.append(feat)

df_color = pd.DataFrame(color_features)
df_color

,mean_saturation,mean_hue,rg_ratio,bg_ratio,imagem
0,169.167450,87.533859,0.541665,1.168660,foto1.jpg
1,69.765778,77.590500,0.876877,1.082426,foto2.jpg
2,183.676651,82.239548,0.572623,1.131857,foto3.jpg
3,91.411606,65.837128,0.897029,0.954200,foto4.jpg
4,184.257828,88.408493,0.491342,1.357932,foto5.jpg
5,54.159653,77.660156,0.957885,1.040412,foto6.jpg
6,108.783630,72.484741,1.145898,0.764622,foto7.jpg
7,115.372162,91.170532,0.956959,1.273186,foto8.jpg
8,148.201370,23.080734,1.141849,0.593962,foto9.jpg
9,131.410538,87.925568,0.717193,1.044025,foto10.jpg


# Abordagem 3 - Estrutura e fontes de luz pontuais


## Resumo
A terceira abordagem tem como objetivo complementar as abordagens baseadas em iluminação global e cor, focando na distribuição espacial do brilho e na presença de fontes de luz artificiais.

## Motivação
A classificação é feita identificando a combinação de fundo escuro + picos intensos isolados, característicos de iluminação artificial noturna.

## Pipeline
1. Leitura da imagem
2. Conversão para escala de cinza
3. Extração de estatísticas globais
4. Classificação baseada em distribuição

## Limitações
Assume-se que cenas noturnas apresentam fontes artificiais de luz (postes, fachadas, faróis).
Em ambientes noturnos pouco iluminados (zonas rurais, estradas isoladas, praias), essa hipótese não se sustenta, podendo resultar em falsos positivos de “dia”.

In [ ]:
# Obtenção das métricas

def features_struct(image, bins=64):
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    _, _, v = cv.split(hsv)

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    edges = cv.Canny(gray, 80, 160)

    mean_brightness = np.mean(v)
    max_brightness  = np.max(v)

    return {
        "mean_brightness": np.mean(v),
        "max_brightness": np.max(v),
        "contrast_ratio": max_brightness / (mean_brightness + 1e-6),
        "bright_ratio": np.mean(v > 220),
        "edge_density": np.mean(edges > 0)
    }

In [ ]:
# Classificação

def classify_struct(structFeat, structStats):

    low_brightness = structFeat["mean_brightness"] < structStats["brightness_low"]

    high_contrast = structFeat["contrast_ratio"] > structStats["contrast_high"]

    low_bright_ratio = structFeat["bright_ratio"] < structStats["bright_mid"]

    low_edge_density = structFeat["edge_density"] < structStats["edge_mid"]
    high_edge_density = structFeat["edge_density"] > structStats["edge_high"]

    # Cena clara
    if not low_brightness:
        return "dia"

    # Cena escura com muita estrutura urbana → dia
    if high_edge_density and low_bright_ratio:
        return "dia"

    # Cena escura com picos isolados de luz → noite
    if high_contrast and low_edge_density:
        return "noite"

    return "dia"


In [ ]:

import pandas as pd

struct_features = []

for i, img in enumerate(images):
    feat = features_struct(cv.resize(img, (256, 256)))
    feat["imagem"] = f"foto{i+1}.jpg"
    struct_features.append(feat)

df_struct = pd.DataFrame(struct_features)
df_struct.describe()
df_struct


,mean_brightness,max_brightness,contrast_ratio,bright_ratio,edge_density,imagem
0,156.956693,255,1.624652,0.129993,0.045541,foto1.jpg
1,143.299697,255,1.779487,0.043304,0.017903,foto2.jpg
2,70.269141,255,3.628904,0.019527,0.047525,foto3.jpg
3,140.455393,255,1.815523,0.110842,0.131842,foto4.jpg
4,80.211561,255,3.179093,0.009307,0.039681,foto5.jpg
5,140.565236,255,1.814104,0.342633,0.156908,foto6.jpg
6,42.946524,255,5.937617,0.028486,0.016161,foto7.jpg
7,178.483639,255,1.428702,0.432986,0.055911,foto8.jpg
8,62.274539,255,4.094771,0.063132,0.036547,foto9.jpg
9,80.472368,255,3.168790,0.023462,0.010168,foto10.jpg


# Classificação final: Tabela
Basta clicar no botão "Executar tudo' para gerar a tabela de comparação dos resultados de cada imagem!

In [ ]:
# Definição das thresh: abordagem 1

illumStats = {
    "brightness_low":  np.percentile(df_illum["mean_brightness"], 30),
    "brightness_mid":  np.percentile(df_illum["mean_brightness"], 50),
    "brightness_high": np.percentile(df_illum["mean_brightness"], 70),

    "saturation_low":  np.percentile(df_illum["mean_saturation"], 30),
    "saturation_mid":  np.percentile(df_illum["mean_saturation"], 50),
    "saturation_high": np.percentile(df_illum["mean_saturation"], 70),

    "shadow_low":      np.percentile(df_illum["p90"] - df_illum["p10"], 30),
    "shadow_mid":      np.percentile(df_illum["p90"] - df_illum["p10"], 50),
    "shadow_high":     np.percentile(df_illum["p90"] - df_illum["p10"], 70),

    "contrast_low":    np.percentile(df_illum["std_brightness"], 30),
    "contrast_mid":    np.percentile(df_illum["std_brightness"], 50),
    "contrast_high":   np.percentile(df_illum["std_brightness"], 70)
}


In [ ]:
# Definição das thresh: abordagem 2

colorStats = {
    "saturation_low": np.percentile(df_color["mean_saturation"], 30),
    "saturation_high": np.percentile(df_color["mean_saturation"], 70),

    "rg_high": df_color["rg_ratio"].quantile(0.85),
    "bg_high": df_color["bg_ratio"].quantile(0.85),

    "hue_low":  df_color["mean_hue"].quantile(0.10),
    "hue_high": df_color["mean_hue"].quantile(0.90),
}


In [ ]:
# Definição das thresh: abordagem 3

structStats = {
    "brightness_low":  np.percentile(df_struct["mean_brightness"], 30),
    "brightness_mid":  np.percentile(df_struct["mean_brightness"], 50),
    "brightness_high": np.percentile(df_struct["mean_brightness"], 70),

    "contrast_low":    np.percentile(df_struct["contrast_ratio"], 30),
    "contrast_mid":    np.percentile(df_struct["contrast_ratio"], 50),
    "contrast_high":   np.percentile(df_struct["contrast_ratio"], 70),

    "bright_low":      np.percentile(df_struct["bright_ratio"], 30),
    "bright_mid":      np.percentile(df_struct["bright_ratio"], 50),
    "bright_high":     np.percentile(df_struct["bright_ratio"], 70),

    "edge_low":        np.percentile(df_struct["edge_density"], 30),
    "edge_mid":        np.percentile(df_struct["edge_density"], 50),
    "edge_high":       np.percentile(df_struct["edge_density"], 70)
}


In [ ]:
# Resultado tabelado

results = []

for i, img in enumerate(images):
    img_resized = cv.resize(img, (256, 256))
    illumFeat = features_illumination(img_resized)
    colorFeat = features_color(img_resized)
    structFeat = features_struct(img_resized)

    results.append({
        "imagem": f"foto{i+1}.jpg",
        "classificacao_iluminacao": classify_illumination(illumFeat, illumStats),
        "classificação_cor": classify_color(colorFeat, colorStats),
        "classificacao_estrutura": classify_struct(structFeat, structStats)
    })

df_results = pd.DataFrame(results)
df_results


,imagem,classificacao_iluminacao,classificação_cor,classificacao_estrutura
0,foto1.jpg,dia,dia,dia
1,foto2.jpg,dia,noite,dia
2,foto3.jpg,noite,dia,dia
3,foto4.jpg,dia,dia,dia
4,foto5.jpg,noite,dia,dia
5,foto6.jpg,dia,noite,dia
6,foto7.jpg,noite,noite,noite
7,foto8.jpg,dia,noite,dia
8,foto9.jpg,dia,dia,noite
9,foto10.jpg,noite,dia,dia


# Conclusão - Análise Crítica
### Abordagem 1
Baseia-se em métricas globais de brilho, saturação e contraste para distinguir dia e noite de forma simples e direta.

Motivada pela diferença natural de iluminação solar entre os períodos.

Funciona bem em cenas abertas, mas falha em dias nublados, contraluz e ambientes urbanos iluminados à noite. Não considera a estrutura espacial da cena nem fontes de luz artificiais.


### Abordagem 2
Explora relações entre canais de cor (como proporções RG e BG) para inferir condições de iluminação.

A motivação é capturar diferenças espectrais entre luz solar e iluminação artificial.

É sensível a balanço de branco, clima e estilo da cena, apresentando desempenho instável em ambientes urbanos ou sob iluminação mista. Não é robusta a variações de câmera.

Não funcionou como o desejado, apresentando falhas críticas na classificação porque não foram métricas ideais para segregação das imagens.


### Abordagem 3
Utiliza brilho global, contraste estrutural, densidade de bordas e concentração de regiões brilhantes para identificar padrões de iluminação artificial e estrutura urbana.

Motivada pela observação de que cenas noturnas apresentam picos localizados de luz, enquanto dias urbanos mantêm alta estrutura visual mesmo com pouco brilho.

Logo, a limitação mais evidente é classificar paisagens noturnas com altos picos de luz.

Pode falhar em noites muito iluminadas ou em cenas com pouca textura.

### Conclusão geral
A maior dificuldade do problema foi identificar que a imagem "foto12.jpg" foi tirada durante o dia; maior empecilho foi a alta densidade de estruturas e pico de luz baixo, fazendo com que as métricas se assemelhassem muito a uma paisagem noturna.

Além de que este ambiente desassocia sobre a iluminação natural, contendo bastante interferência para classificação. E com correção para a classificação correta, interfere diretamente sobre a decisão de paisagens noturnas com alta densidade de luz artificial.

A abordagem 2 foi a que mais não foi eficiente e não demonstrou confiança na classificação diante das métricas escolhidas.

Vale ressaltar que a abordagem 3 apresentou ser mais sensível a iluminação artificial em paisagens noturnas.

Além disso, a abordagem 1 mostrou mais "conservadorismo" referente a classificação diante do horário e clima da paisagem - em momentos de anoitecer/amanhecer e/ou nublados; mantinha para classificação "noite'. Já para a abordagem 3, nestes aspectos desempenhou sensibilidade a presença de luz devido à capacidade de distribuição da iluminação.

Logo, a abordagem que chegou mais próximo a realidade foi a 1, referente a métricas de iluminação global da imagem.